In [ ]:
# LSTM for Stock Market Prediction with Gann Features (multiple tickers, no scaling)

import math
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

np.random.seed(42)
tf.random.set_seed(42)

# ----------------- Config -----------------
TICKERS = ["AAPL", "MSFT", "GOOG"]   # List of tickers
START = "2015-01-01"
END = "2025-01-01"
SEQ_LEN = 21
EPOCHS = 30
BATCH_SIZE = 32
# ------------------------------------------

import numpy as np

def add_gann_features(df, step=0.25):
    df = df.copy()

    def _gann_calc(price):
        p = float(price)
        root_price = np.sqrt(p)
        res = (root_price + step) ** 2
        sup = (root_price - step) ** 2
        return res, sup

    # Find the Close column dynamically
    close_cols = [col for col in df.columns if col[0] == 'Close']
    close_col = close_cols[0]  # Get first Close column
    ticker = close_col[1]  # Extract ticker name (AAPL, MSFT, etc.)

    gann_values = df[close_col].apply(_gann_calc)
    df[('GannRes', ticker)] = [x[0] for x in gann_values]
    df[('GannSup', ticker)] = [x[1] for x in gann_values]

    return df


def create_sequences(arr, seq_len, target_col_idx=3):
    X, y = [], []
    for i in range(len(arr) - seq_len):
        X.append(arr[i:i+seq_len])
        y.append(arr[i+seq_len, target_col_idx])
    return np.array(X), np.array(y)

def build_lstm(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=input_shape),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

for ticker in TICKERS:
    print(f"\n===== Running LSTM + Gann for {ticker} =====")

    # 1) Download data
    df = yf.download(ticker, start=START, end=END, progress=False)
    df = df[['Open', 'High', 'Low', 'Close']].round(2).dropna()

    # 2) Add Gann features
    df = add_gann_features(df)


    # 3) Sequence creation
    values = df.values
    X, y = create_sequences(values, SEQ_LEN, 3)

    # Train-test split
    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]

    # 4) Model
    model = build_lstm((SEQ_LEN, X.shape[2]))
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)

    # 5) Evaluate
    pred = model.predict(X_test).ravel()
    rmse = math.sqrt(mean_squared_error(y_test, pred))
    mae = mean_absolute_error(y_test, pred)
    mape = mean_absolute_percentage_error(y_test, pred)

    print(f"{ticker} → RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}")


===== Running LSTM + Gann for AAPL =====


/tmp/ipython-input-3494779776.py:68: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
AAPL → RMSE: 103.1703, MAE: 99.3700, MAPE: 0.5161

===== Running LSTM + Gann for MSFT =====


/tmp/ipython-input-3494779776.py:68: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
MSFT → RMSE: 284.0307, MAE: 277.2249, MAPE: 0.7531

===== Running LSTM + Gann for GOOG =====


/tmp/ipython-input-3494779776.py:68: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
GOOG → RMSE: 70.6104, MAE: 65.0326, MAPE: 0.4351
